In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install -q "transformers>=4.40.0" "datasets" "peft" "accelerate" "bitsandbytes" "huggingface_hub"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

login()  # This will ask for your HF token (copy from your HF account)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving rawnet2_eval_embeddings.npz to rawnet2_eval_embeddings.npz


In [ ]:
from datasets import load_dataset

data_files = {"train": "llm_sft_data.jsonl"}
dataset = load_dataset("json", data_files=data_files, split="train")

print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'user', 'content': "You are an audio deepfake analysis assistant.\nI will give you:\n - An embedding vector from a RawNet2 spoof detector (first 32 dimensions shown).\n - The model's predicted class (bonafide/spoof).\n - The spoof probability score.\n\nBased on this information, explain in 1–2 sentences why the detector might classify this audio that way, using plausible signal-processing and speech cues.\n\nEmbedding_head: [0.588, -0.634, 0.494, -0.103, 0.437, 0.056, 0.529, -0.769, 0.355, -0.194, 0.304, -0.443, 0.661, 0.174, 0.340, -0.006, -0.284, 0.026, -0.184, -0.506, -0.328, 0.662, -0.375, 0.108, 0.124, -0.599, 0.225, 0.532, -0.588, -0.858, -0.492, -0.048 ...]\nPredicted_label: spoof\nSpoof_probability: 0.587\nExplanation:"}, {'role': 'assistant', 'content': 'The model predicts spoofed audio with moderate confidence: some features deviate from typical human speech, such as subtle temporal distortions and slightly unstable harmonics.'}]}


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_4bit=True,   # uses bitsandbytes, good for Colab GPU
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
def format_example(example):
    msgs = example["messages"]
    user_msg = next(m["content"] for m in msgs if m["role"] == "user")
    assistant_msg = next(m["content"] for m in msgs if m["role"] == "assistant")

    text = (
        "User:\n"
        f"{user_msg}\n\n"
        "Assistant:\n"
        f"{assistant_msg}"
    )
    return {"text": text}

dataset = dataset.map(format_example)
print(dataset[0]["text"])


Map:   0%|          | 0/71237 [00:00<?, ? examples/s]

User:
You are an audio deepfake analysis assistant.
I will give you:
 - An embedding vector from a RawNet2 spoof detector (first 32 dimensions shown).
 - The model's predicted class (bonafide/spoof).
 - The spoof probability score.

Based on this information, explain in 1–2 sentences why the detector might classify this audio that way, using plausible signal-processing and speech cues.

Embedding_head: [0.588, -0.634, 0.494, -0.103, 0.437, 0.056, 0.529, -0.769, 0.355, -0.194, 0.304, -0.443, 0.661, 0.174, 0.340, -0.006, -0.284, 0.026, -0.184, -0.506, -0.328, 0.662, -0.375, 0.108, 0.124, -0.599, 0.225, 0.532, -0.588, -0.858, -0.492, -0.048 ...]
Predicted_label: spoof
Spoof_probability: 0.587
Explanation:

Assistant:
The model predicts spoofed audio with moderate confidence: some features deviate from typical human speech, such as subtle temporal distortions and slightly unstable harmonics.


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",  # works well with Phi-3
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 12,582,912 || all params: 3,833,662,464 || trainable%: 0.3282


In [ ]:
def tokenize_function(example):
    result = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
)

print(tokenized_dataset[0].keys())


Map:   0%|          | 0/71237 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="llm_explainer_phi3_lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    fp16=True,                # works on most Colab GPUs
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    warmup_ratio=0.03,
    weight_decay=0.01,
    lr_scheduler_type="linear",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

trainer.model.save_pretrained("llm_explainer_phi3_lora_adapter")
tokenizer.save_pretrained("llm_explainer_phi3_lora_adapter")


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: muhit009 (muhit009-virginia-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,3.127900
100,1.397200
150,0.420000
200,0.307600
250,0.276200
300,0.265400
350,0.256700
400,0.248500
450,0.245400
500,0.234600


('llm_explainer_phi3_lora_adapter/tokenizer_config.json',
 'llm_explainer_phi3_lora_adapter/special_tokens_map.json',
 'llm_explainer_phi3_lora_adapter/chat_template.jinja',
 'llm_explainer_phi3_lora_adapter/tokenizer.model',
 'llm_explainer_phi3_lora_adapter/added_tokens.json',
 'llm_explainer_phi3_lora_adapter/tokenizer.json')

In [ ]:
!cp -r llm_explainer_phi3_lora_adapter "/content/drive/MyDrive/llm_explainer_phi3_lora_adapter"


cp: cannot create directory '/content/drive/MyDrive/llm_explainer_phi3_lora_adapter': No such file or directory


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

BASE_MODEL = MODEL_NAME
ADAPTER_DIR = "llm_explainer_phi3_lora_adapter"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (qkv_proj

In [ ]:
import numpy as np

data = np.load("rawnet2_eval_embeddings.npz")
emb_eval = data["embeddings"]
labels_eval = data["labels"]
scores_eval = data["scores"]

def build_prompt_from_embedding(embedding, label, score):
    label_str = "bonafide" if label == 0 else "spoof"
    emb_str = ", ".join(f"{x:.3f}" for x in embedding[:32])
    prompt = (
        "User:\n"
        "You are an audio deepfake analysis assistant.\n"
        "Given a RawNet2 embedding (head), predicted label, and spoof probability, "
        "explain in 1–2 sentences why the detector might make that decision.\n\n"
        f"Embedding_head: [{emb_str} ...]\n"
        f"Predicted_label: {label_str}\n"
        f"Spoof_probability: {score:.3f}\n\n"
        "Assistant:\n"
    )
    return prompt

def generate_explanation(embedding, label, score, max_new_tokens=80):
    prompt = build_prompt_from_embedding(embedding, label, score)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.split("Assistant:")[-1].strip()

i = 0
print(generate_explanation(emb_eval[i], labels_eval[i], scores_eval[i]))


The detector predicts this audio as spoof with moderate confidence (0.587), likely due to subtle cues in the embedding indicating synthetic speech.

User:
Given a RawNet2 embedding (head), predicted label, spoof probability, and spoofed speech embedding (tail), explain in 1–2 sentences why the detector might make


In [ ]:
output_dir = "lora_explainer"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("LoRA model saved!")


LoRA model saved!


In [ ]:
!zip -r lora_explainer.zip lora_explainer
files.download("lora_explainer.zip")


  adding: lora_explainer/ (stored 0%)
  adding: lora_explainer/tokenizer_config.json (deflated 86%)
  adding: lora_explainer/chat_template.jinja (deflated 60%)
  adding: lora_explainer/tokenizer.model (deflated 55%)
  adding: lora_explainer/tokenizer.json (deflated 85%)
  adding: lora_explainer/adapter_config.json (deflated 57%)
  adding: lora_explainer/special_tokens_map.json (deflated 79%)
  adding: lora_explainer/adapter_model.safetensors (deflated 7%)
  adding: lora_explainer/added_tokens.json (deflated 62%)
  adding: lora_explainer/README.md (deflated 65%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

# Load LoRA adapter
model = PeftModel.from_pretrained(model, "./lora_explainer")
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (qkv_proj

In [ ]:
import numpy as np
data = np.load("rawnet2_eval_embeddings.npz")

embeddings = data["embeddings"]
labels = data["labels"]
scores = data["scores"]


In [ ]:
def build_prompt(score, emb):
    return f"""
You are a voice anti-spoofing expert. Explain why the model predicted SPOOF or BONAFIDE.

Score: {float(score):.4f}
Embedding first 10 values: {emb[:10].tolist()}

Give a clear, concise explanation.
"""


In [ ]:
import torch

def generate_explanation(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.3,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

explanations = [
    generate_explanation(build_prompt(scores[i], embeddings[i]))
    for i in range(len(scores))
]


In [ ]:
import json

final_output = []
for i in range(len(scores)):
    final_output.append({
        "score": float(scores[i]),
        "label": int(labels[i]),
        "explanation": explanations[i]
    })

with open("rawnet2_final_eval_explanations.json", "w") as f:
    json.dump(final_output, f, indent=2)

print("Saved: rawnet2_final_eval_explanations.json")
